In [ ]:
from random import choice, expovariate
from copy import copy
from typing import Tuple, Callable
import numpy as np
import datetime
from datetime import timedelta

def successors_by_predecessors(predecessors: list[list[int]]):
    size = len(predecessors)
    return [[succ for succ in range(size) if i in predecessors[succ]] for i in range(size)]


def calculate_critical_times(
        duration: list[int],
        predecessors: list[list[int]],
        successors: list[list[int]] = None) -> Tuple[list[int], list[int]]:
    """Расчёт ранних времён начала (EST) и поздних времён завершения (LFT)."""

    if successors is None:
        successors = successors_by_predecessors(predecessors)

    if len(duration) != len(predecessors) or len(predecessors) != len(successors):
        raise ValueError("Invalid data lengths")

    earliest_start = [0 for _ in duration]
    latest_finish = [0 for _ in duration]

    def _calc_earliest_start(idx: int) -> int:
        if predecessors[idx]:
            earliest_start[idx] = max(_calc_earliest_start(p) + duration[p] for p in predecessors[idx])
        return earliest_start[idx]

    def _calc_latest_finish(idx: int) -> int:
        # позднее завершение "финальной" работы приравниваем к её EST
        if idx == len(successors) - 1:
            latest_finish[idx] = earliest_start[idx]
            return latest_finish[idx]

        if successors[idx]:
            latest_finish[idx] = min(_calc_latest_finish(s) - duration[s] for s in successors[idx])
        else:
            # если задача без последователей — прижимаем к концу проекта
            latest_finish[idx] = earliest_start[-1]
        return latest_finish[idx]

    _calc_earliest_start(len(duration) - 1)
    _calc_latest_finish(0)

    return earliest_start, latest_finish


class TimeCapacityNode:
    """Связный список моментов времени и остаточных запасов ресурсов."""
    def __init__(self, time: int, capacity: list[int]):
        self.time = time
        self.capacity = capacity
        self.next = None
        self.prev = None

    def insert_after(self, time: int) -> 'TimeCapacityNode':
        if time <= self.time:
            raise ValueError("Invalid time")

        new_node = self.__class__(time, copy(self.capacity))
        new_node.prev = self
        new_node.next = self.next
        if self.next:
            self.next.prev = new_node
        self.next = new_node
        return new_node

    def find_first(self, time: int) -> 'TimeCapacityNode':
        node = self
        while node.time < time:
            if node.next:
                node = node.next
            else:
                return node
        return node.prev

    def enough_resources(self, demand: list[int]) -> bool:
        return all(self.capacity[i] >= demand[i] for i in range(len(self.capacity)))

    def consume(self, demand: list[int]) -> None:
        for i in range(len(self.capacity)):
            self.capacity[i] -= demand[i]


class ActivityListDecoder:
    """SSGS-декодер Activity List в расписание при ресурсных ограничениях."""
    def decode(
            self,
            activity_list: list[int],
            duration: list[int],
            predecessors: list[list[int]],
            renewable_demands: list[list[int]],
            renewable_capacity: list[int]) -> list[int]:

        count = len(activity_list)
        root_node = TimeCapacityNode(0, copy(renewable_capacity))
        starts = [0] * count
        finish_nodes = [None] * count
        finish_nodes[0] = root_node

        for i in activity_list:
            # старт не раньше завершения всех предшественников
            start_node = root_node
            for pred in predecessors[i]:
                if not finish_nodes[pred]:
                    raise ValueError("Invalid activity list: predecessor not scheduled")
                if finish_nodes[pred].time > start_node.time:
                    start_node = finish_nodes[pred]

            start_node, last_node, finish_node, finish_time = self._find_position(
                start_node, duration[i], renewable_demands[i]
            )

            starts[i] = start_node.time

            if not finish_node or finish_node.time != finish_time:
                finish_node = last_node.insert_after(finish_time)

            finish_nodes[i] = finish_node
            self._consume(start_node, finish_node, renewable_demands[i])

        return starts

    def _consume(self, start_node: TimeCapacityNode, finish_node: TimeCapacityNode, demand: list[int]) -> None:
        node = start_node
        while node != finish_node:
            node.consume(demand)
            node = node.next

    def _find_position(
            self, start_node: TimeCapacityNode, duration: int, demand: list[int]
    ) -> Tuple[TimeCapacityNode, TimeCapacityNode, TimeCapacityNode, int]:

        if duration == 0:
            return (start_node, start_node, start_node, start_node.time)

        finish_time = start_node.time + duration
        t = start_node.find_first(finish_time)
        last_node = t
        t_test = start_node

        while t != t_test.prev:
            if t.enough_resources(demand):
                t = t.prev
            else:
                start_node = t.next
                finish_time = start_node.time + duration
                if last_node.next:
                    t_test = last_node.next
                    last_node = t_test.find_first(finish_time)
                    t = last_node
                else:
                    break

        return (start_node, last_node, last_node.next, finish_time)


class ActivityListSampler:
    """Генератор Activity List (случайно / по эвристикам)."""
    def __init__(self, predecessors: list[list[int]], successors: list[list[int]] = None) -> None:
        self.predecessors = predecessors
        self.size = len(predecessors)
        if successors is None:
            successors = successors_by_predecessors(predecessors)
        self.successors = successors

    def _generate(self, func: Callable = None) -> list[int]:
        result = []
        remain_predecessors = [set(pred) for pred in self.predecessors]
        ready_set = [i for i in range(self.size) if not self.predecessors[i]]

        for _ in range(self.size):
            if not ready_set:
                raise ValueError("Incorrect project network (cycle or wrong deps)")

            next_activity = func(ready_set) if func else choice(ready_set)
            ready_set.remove(next_activity)
            result.append(next_activity)

            for successor in self.successors[next_activity]:
                remain_predecessors[successor].remove(next_activity)
                if not remain_predecessors[successor]:
                    ready_set.append(successor)

        return result

    def generate_random(self) -> list[int]:
        return self._generate()

    def generate_by_min_rule(self, rule: Callable) -> list[int]:
        def func(data: list[int]):
            return min(data, key=rule)
        return self._generate(func)

    def generate_by_max_rule(self, rule: Callable) -> list[int]:
        def func(data: list[int]):
            return max(data, key=rule)
        return self._generate(func)

    # SLK: общий резерв (чем меньше, тем критичнее)
    def generate_by_SLK(self, lft: list[int], est: list[int], duration: list[int]) -> list[int]:
        def slk(j: int) -> int:
            lst = lft[j] - duration[j]
            return lst - est[j]
        return self.generate_by_min_rule(slk)

    # FREE: свободный резерв
    def generate_by_FREE(self, est: list[int], duration: list[int]) -> list[int]:
        def free(j: int) -> int:
            succs = self.successors[j]
            if not succs:
                return 0
            return min(est[s] for s in succs) - (est[j] + duration[j])
        return self.generate_by_min_rule(free)

    # LST: позднее время начала
    def generate_by_LST(self, lft: list[int], duration: list[int]) -> list[int]:
        return self.generate_by_min_rule(lambda j: lft[j] - duration[j])

    # LFT: позднее время конца
    def generate_by_LFT(self, lft: list[int]) -> list[int]:
        return self.generate_by_min_rule(lambda j: lft[j])

    # EXP: случайная эвристика (экспоненциальные веса)
    def generate_by_EXP(self, lambda_param: float = 1.0) -> list[int]:
        def func(data: list[int]):
            weights = [expovariate(lambda_param) for _ in range(len(data))]
            return data[int(np.argmin(weights))]
        return self._generate(func)

EST: [0, 0, 3, 7, 0, 7, 12, 17, 0, 0, 0, 21, 17, 21, 21, 0, 21, 0, 0, 0, 0, 7, 11, 26]
LFT: [-4, -1, 3, 15, 26, 8, 13, 17, 26, 26, 26, 23, 19, 23, 23, 26, 22, 26, 26, 23, 26, 20, 23, 26]

=== СРАВНЕНИЕ ЭВРИСТИК ===
RANDOM makespan =  90 раб.дней
LFT    makespan =  87 раб.дней
LST    makespan =  87 раб.дней
SLK    makespan =  87 раб.дней
FREE   makespan =  90 раб.дней
EXP    makespan =  90 раб.дней

Лучшая эвристика: LFT
Лучший makespan: 87 рабочих дней
Activity List: [0, 1, 2, 5, 6, 3, 7, 12, 21, 16, 19, 11, 14, 13, 22, 4, 8, 9, 10, 17, 18, 20, 15, 23]

=== ДЕТАЛЬНЫЙ КАЛЕНДАРНЫЙ ГРАФИК ===
Задача  1 | Экран авторизации пользователя                          | 3дн | (раб: 0-3)
Задача  2 | Аутентификация и хранение сессии                        | 4дн | (раб: 3-7)
Задача  3 | Экран управления ролями и правами                       | 4дн | (раб: 17-21)
Задача  4 | Назначение ролей пользователям                          | 3дн | (раб: 52-55)
Задача  5 | Приём логов от внешних источников      

In [ ]:
# Нумерация
TASKS = {
    0: "Старт (фиктивная)",
    1: "Экран авторизации пользователя",
    2: "Аутентификация и хранение сессии",
    3: "Экран управления ролями и правами",
    4: "Назначение ролей пользователям",
    5: "Приём логов от внешних источников",
    6: "Сохранение и хранение логов",
    7: "Экран просмотра логов",
    8: "Пагинация и сортировка логов",
    9: "Фильтрация логов по параметрам",
    10: "Полнотекстовый поиск по логам",
    11: "Детальный просмотр события",
    12: "Настройка уведомлений о событиях",
    13: "Механизм отправки уведомлений",
    14: "Экспорт логов",
    15: "Выбор формата и периода экспорта",
    16: "Статистика и аналитика событий",
    17: "Визуализация активности и метрик",
    18: "Агрегирование по периодам",
    19: "Анализ аномалий (ML)",
    20: "Просмотр результатов аномалий",
    21: "Журнал действий администраторов",
    22: "Экран просмотра журнала действий",
    23: "Финальное развёртывание системы"
}

# Предшественники
predecessors = [
    [],                 # 0
    [0],                # 1
    [1],                # 2
    [2],                # 3
    [3],                # 4
    [2],                # 5
    [5],                # 6
    [6],                # 7
    [7],                # 8
    [7],                # 9
    [7],                # 10
    [7],                # 11
    [3, 6],             # 12
    [12],               # 13
    [7],                # 14
    [14],               # 15
    [7],                # 16
    [16],               # 17
    [16],               # 18
    [6],                # 19
    [19],               # 20
    [2],                # 21
    [21],               # 22
    [11, 13, 14, 16, 22]# 23
]

# Длительности (рабочие дни)
durations = [
    0,  # 0 фиктивная
    3,  # 1 экран авторизации
    4,  # 2 сессия/аутентификация
    4,  # 3 роли и права
    3,  # 4 назначение ролей
    5,  # 5 приём логов
    5,  # 6 хранение логов
    4,  # 7 просмотр логов
    3,  # 8 пагинация/сортировка
    4,  # 9 фильтрация
    5,  # 10 поиск
    3,  # 11 детали события
    4,  # 12 настройки уведомлений
    4,  # 13 отправка уведомлений
    4,  # 14 экспорт логов
    2,  # 15 формат/период экспорта
    5,  # 16 аналитика
    4,  # 17 визуализация
    4,  # 18 агрегация
    7,  # 19 аномалии (ML)
    3,  # 20 просмотр аномалий
    4,  # 21 журнал действий
    3,  # 22 экран журнала
    3   # 23 финальный деплой
]

assert len(predecessors) == len(durations) == 24

renewable_capacities = [
    1,  # PM
    1,  # Analyst
    1,  # QA
    2,  # Backend
    1,  # Frontend
    1,  # DevOps
    1   # ML/Data
]

# Ресурсные требования задач:
# 0/1/2 — сколько людей каждого типа нужно одновременно на задаче.
renewable_demands = [
    [0,0,0,0,0,0,0],  # 0 фиктивная

    [1,1,1,0,1,0,0],  # 1 экран авторизации: PM+Analyst+QA+Frontend
    [1,1,1,1,0,0,0],  # 2 аутентификация/сессия: PM+Analyst+QA+Backend
    [1,1,0,1,1,0,0],  # 3 роли/права: PM+Analyst+Backend+Frontend
    [0,1,0,1,0,0,0],  # 4 назначение ролей: Analyst+Backend

    [0,1,1,2,0,1,0],  # 5 приём логов: Analyst+QA+2BE+DevOps
    [0,1,1,2,0,1,0],  # 6 хранение логов: Analyst+QA+2BE+DevOps
    [0,1,1,1,1,0,0],  # 7 просмотр логов: Analyst+QA+BE+Frontend
    [0,1,1,1,1,0,0],  # 8 пагинация/сортировка: Analyst+QA+BE+Frontend
    [0,1,1,1,1,0,0],  # 9 фильтрация: Analyst+QA+BE+Frontend
    [0,1,1,1,1,0,0],  # 10 поиск: Analyst+QA+BE+Frontend
    [0,1,1,1,1,0,0],  # 11 детали события: Analyst+QA+BE+Frontend

    [1,1,1,1,1,0,0],  # 12 настройки уведомлений: PM+Analyst+QA+BE+FE
    [0,0,1,1,0,1,0],  # 13 отправка уведомлений: QA+BE+DevOps
    [0,1,1,1,1,0,0],  # 14 экспорт логов: Analyst+QA+BE+FE
    [0,1,0,1,1,0,0],  # 15 формат/период: Analyst+BE+FE

    [1,1,1,1,1,0,0],  # 16 аналитика: PM+Analyst+QA+BE+FE
    [0,1,1,1,1,0,0],  # 17 визуализация: Analyst+QA+BE+FE
    [0,1,1,1,0,0,0],  # 18 агрегация: Analyst+QA+BE

    [0,1,1,1,0,0,1],  # 19 аномалии: Analyst+QA+BE+ML
    [0,1,1,1,1,0,0],  # 20 просмотр аномалий: Analyst+QA+BE+FE

    [0,1,1,1,0,0,0],  # 21 журнал действий: Analyst+QA+BE
    [0,1,1,1,1,0,0],  # 22 экран журнала: Analyst+QA+BE+FE

    [1,1,1,1,1,1,0],  # 23 финальный деплой: PM+Analyst+QA+BE+FE+DevOps
]

assert len(renewable_demands) == 24
assert all(len(d) == len(renewable_capacities) for d in renewable_demands)


successors = successors_by_predecessors(predecessors)
earliest_start, latest_finish = calculate_critical_times(durations, predecessors, successors)

print("EST:", earliest_start)
print("LFT:", latest_finish)

sampler = ActivityListSampler(predecessors, successors)
decoder = ActivityListDecoder()

solutions = {
    "RANDOM": sampler.generate_random(),
    "LFT":    sampler.generate_by_LFT(latest_finish),
    "LST":    sampler.generate_by_LST(latest_finish, durations),
    "SLK":    sampler.generate_by_SLK(latest_finish, earliest_start, durations),
    "FREE":   sampler.generate_by_FREE(earliest_start, durations),
    "EXP":    sampler.generate_by_EXP(lambda_param=1.0),
}

best_makespan = float("inf")
best_name = None
best_alist = None
best_starts = None

print("\n=== СРАВНЕНИЕ ЭВРИСТИК ===")
for name, alist in solutions.items():
    try:
        starts = decoder.decode(alist, durations, predecessors, renewable_demands, renewable_capacities)
        makespan = max(starts[i] + durations[i] for i in range(len(durations)))
        print(f"{name:6} makespan = {makespan:3} раб.дней")
        if makespan < best_makespan:
            best_makespan = makespan
            best_name = name
            best_alist = alist
            best_starts = starts
    except Exception as e:
        print(f"{name:6} -> ошибка: {e}")

print("\nЛучшая эвристика:", best_name)
print("Лучший makespan:", best_makespan, "рабочих дней")
print("Activity List:", best_alist)


def work_days_to_calendar(start_date: datetime.date, work_days: int) -> datetime.date:
    """Конвертация 'рабочих дней от старта' -> календарная дата по графику 5/2."""
    full_weeks = work_days // 5
    rem = work_days % 5
    date = start_date + timedelta(days=full_weeks * 7 + rem)
    while date.weekday() >= 5:
        date += timedelta(days=1)
    return date


def calculate_calendar_schedule(starts: list[int], durations: list[int], start_date: datetime.date):
    schedule = []
    for i in range(len(durations)):
        if durations[i] == 0:
            continue
        ws = starts[i]
        wf = starts[i] + durations[i]
        cs = work_days_to_calendar(start_date, ws)
        cf = work_days_to_calendar(start_date, wf)
        schedule.append((i, cs, cf, durations[i], ws, wf))
    return schedule


PROJECT_START = datetime.date(2026, 1, 12)
calendar_schedule = calculate_calendar_schedule(best_starts, durations, PROJECT_START)

print("\n=== ДЕТАЛЬНЫЙ КАЛЕНДАРНЫЙ ГРАФИК ===")
for task_id, cs, cf, d, ws, wf in calendar_schedule:
    print(f"Задача {task_id:2} | {TASKS[task_id]:55} | {d}дн | (раб: {ws}-{wf})")

project_finish = work_days_to_calendar(PROJECT_START, int(best_makespan))
print("\nПлановое окончание проекта:", project_finish)
print("Итого:", best_makespan, "рабочих дней")


def finish_day(task_id: int) -> int:
    return best_starts[task_id] + durations[task_id]

milestones = {
    "Начало проекта": 0,
    "Готова авторизация (1-2)": max(finish_day(1), finish_day(2)),
    "Готово ядро ролей (3-4)": max(finish_day(3), finish_day(4)),
    "Готов сбор и хранение логов (5-6)": max(finish_day(5), finish_day(6)),
    "Готов базовый UI логов (7-11)": max(finish_day(7), finish_day(11)),
    "Готовы уведомления (12-13)": max(finish_day(12), finish_day(13)),
    "Готов экспорт (14-15)": max(finish_day(14), finish_day(15)),
    "Готова аналитика (16-18)": max(finish_day(16), finish_day(18)),
    "Готов ML-анализ (19-20)": max(finish_day(19), finish_day(20)),
    "Готов аудит админов (21-22)": max(finish_day(21), finish_day(22)),
    "Завершение проекта (23)": finish_day(23),
}

print("\n=== КЛЮЧЕВЫЕ ВЕХИ ===")
for name, wd in milestones.items():
    print(f"{name:35}: {work_days_to_calendar(PROJECT_START, int(wd))} (раб.день {int(wd)})")
